In [1]:
# Import libraries
import pandas as pd
from pandas import DataFrame
import numpy as np
pd.set_option('display.max_columns', None)  

1) UMLS_id = UMLS concept id as found on the drug label (UMLS - universal medical language system)
2) concept_name = side effect
3) detection_method = method of detection (NLP_indication/NLP_precondition/text_mention)
the authors used NLP on drug label inserts to find the side effects for a particular drug
4) concept_type = MedDRA concept type (LLT= lowest level term, PT= preferred term
in few cases, the term is neither LLT nor PT
NOTE: from medra.tsv file (SIDER website), PT covers all the different forma of a condition.
eg. the PT, abdominal distension covers the different forms namely abdominal distension, distended abdomen, swollen abdomen, swelling abdomen or swelling abd
5) UMLS_id_medra = UMLS concept id for MedDRA term
6) medra_conceptname= MedDRA concept name
IN meddra_all_indications.tsv.gz= all side effects found on the lables are given as LLT. there is atleast one PT for every LLT, but sometimes the PT is the same as the LLT

What is selected for analysis from meddra_df or meddra_all_indications.tsv?
1) stitich_id, concept_name

In [43]:
# Import the datafiles- snap dataset from stanford and SIDER database
#MEDRA is a dictionary used by SIDER to extract side effects info from insert labels
meddra_df = pd.read_table('data/meddra_all_indications.tsv', sep='\t', \
                          names=["stitch_id", "UMLS_id", "detection_method", "concept_name", \
                                 "concept_type", "UMLS_id_medra", "medra_conceptname"])


In [44]:
meddra_df.head(10)
#concept_name and medra_conceptname is indication for which a drug is used.

,stitch_id,UMLS_id,detection_method,concept_name,concept_type,UMLS_id_medra,medra_conceptname
0,CID100000085,C0015544,text_mention,Failure to Thrive,LLT,C0015544,Failure to thrive
1,CID100000085,C0015544,text_mention,Failure to Thrive,PT,C0015544,Failure to thrive
2,CID100000085,C0020615,text_mention,Hypoglycemia,LLT,C0020615,Hypoglycaemia
3,CID100000085,C0020615,text_mention,Hypoglycemia,PT,C0020615,Hypoglycaemia
4,CID100000085,C0022661,NLP_indication,"Kidney Failure, Chronic",LLT,C0022661,Renal failure chronic
5,CID100000085,C0022661,NLP_indication,"Kidney Failure, Chronic",PT,C0022661,Renal failure chronic
6,CID100000085,C0025521,NLP_indication,Inborn Errors of Metabolism,LLT,C0025521,Inborn error of metabolism
7,CID100000085,C0025521,NLP_indication,Inborn Errors of Metabolism,PT,C0025521,Inborn error of metabolism
8,CID100000085,C0026827,text_mention,Muscle hypotonia,LLT,C0026827,Hypotonia
9,CID100000085,C0026827,text_mention,Muscle hypotonia,PT,C0026827,Hypotonia


In [4]:
meddra_df.shape

(30835, 7)

In [5]:
#drop unnecessary columns
meddra_df=meddra_df[meddra_df.concept_type=='PT']
meddra_df=meddra_df.drop(['detection_method', 'concept_type', 'UMLS_id_medra', \
                         'medra_conceptname'], axis=1)

#meddra_all_label_indications.tsv is not needed sincd the only column that is extra in this
#file is the source label which is the first column
meddra_label_df =pd.read_table('data/meddra_all_label_indications.tsv', sep='\t', \
                               names=["source_label", "stitch_id1", "stitch_id2","UMLS_id", \
                                      "detection_method", "concept_name", \
                                 "concept_type", "UMLS_id_medra", "medra_conceptname"])
#What is the difference between stitch_id1 and "stitch_id2"?
#there is only a difference of 1 digit- stitch_id1 is CID1... and stitch_id2 is CID0...
#CIDs / CID0... - this is a stereo-specific compound, and the suffix is the 
#PubChem compound id.
#CIDm / CID1... - this is a "flat" compound, i.e. with merged stereo-isomers
#The suffix (without the leading "1") is the PubChem compound id.

meddra_label_df.head()

meddra_label_df.shape

#medra_label_big is file in which concept name is repeated since there are variants for 
#the concept name
#side effects or concept_name- eg. abdominal pain can be gastrointestinal pain. 

In [9]:
#medra_label_big=pd.read_table('data/meddra_all_label_se.tsv', sep='\t', \
                               #names=["source_label", "stitch_id1", "stitch_id2","UMLS_id", \
                                 #"concept_type", "UMLS_id2", "concept_name"])


In [10]:
#medra_label_big.shape

In [11]:
#medra_label_big.head()

1) cells not needed : detection_method, UMLS_id
2)UMLS id - concept unique identifier for the concept_name. for eg.if you search for C0016658	UMLS_id, it will show concept_name associated with it is fracture.  (https://ncim.nci.nih.gov/ncimbrowser/ConceptReport.jsp?dictionary=NCI%20Metathesaurus&code=C00166580)

In [45]:
#meddra.tsv has name of the side effect
side_effect=pd.read_table('data/meddra.tsv', sep='\t', \
                          names=["umls_id", "meddra_id", "concept_type", "side_effect"])

In [47]:
side_effect=side_effect[side_effect.meddra_id=='PT']

In [48]:
side_effect.head()

,umls_id,meddra_id,concept_type,side_effect
1,C0000727,PT,10000647,Acute abdomen
12,C0000731,PT,10000060,Abdominal distension
17,C0000734,PT,10000077,Abdominal mass
22,C0000735,PT,10060925,Abdominal neoplasm
27,C0000737,PT,10000081,Abdominal pain


In [50]:
side_effect.shape

(20307, 4)

In [ ]:
sid

In [12]:
bio_decagon=pd.read_csv('data/bio-decagon-combo.csv')

In [13]:
bio_decagon.head()

,STITCH 1,STITCH 2,Polypharmacy Side Effect,Side Effect Name
0,CID000002173,CID000003345,C0151714,hypermagnesemia
1,CID000002173,CID000003345,C0035344,retinopathy of prematurity
2,CID000002173,CID000003345,C0004144,atelectasis
3,CID000002173,CID000003345,C0002063,alkalosis
4,CID000002173,CID000003345,C0004604,Back Ache


In [14]:
bio_decagon.shape

(4649441, 4)

In [ ]:
#merge bio_decagon with drug_names

In [15]:
drug_names=pd.read_table('data/drug_names.tsv', sep='\t', header=None, names=["stitch_code", "drug_name"])

In [16]:
drug_names.head()

,stitch_code,drug_name
0,CID100000085,carnitine
1,CID100000119,gamma-aminobutyric
2,CID100000137,5-aminolevulinic
3,CID100000143,leucovorin
4,CID100000146,5-methyltetrahydrofolate


In [17]:
drug_names.shape

(1430, 2)

In [18]:
# merge the drug names with meddra_df (meddra_all_indications data) on the stitch_id
drug_meddra=pd.merge(drug_names, meddra_df, left_on="stitch_code", right_on="stitch_id")

In [19]:
drug_meddra.head()

,stitch_code,drug_name,stitch_id,concept_name
0,CID100000085,carnitine,CID100000085,Failure to Thrive
1,CID100000085,carnitine,CID100000085,Hypoglycemia
2,CID100000085,carnitine,CID100000085,"Kidney Failure, Chronic"
3,CID100000085,carnitine,CID100000085,Inborn Errors of Metabolism
4,CID100000085,carnitine,CID100000085,Muscle hypotonia


In [20]:
drug_meddra.shape

(16037, 4)

In [21]:
#drop the stitch_id since it is same as stitch_code
drug_meddra=drug_meddra.drop(columns=['stitch_id'])

In [22]:
#How many drugs?
# Which drugs have the maximum side effects?
#How many side effects for each drugs?


In [23]:
drug_meddra.nunique()

stitch_code     1360
drug_name       1284
concept_name    2704
dtype: int64

In [24]:
drug_atc=pd.read_table('data/drug_atc.tsv', sep='\t', header=None, names=["stitch_code", "atc_code"])

In [41]:
#ATC code can be used to find the drug name from SIDER database
#IS THERE ANOTHER DATABASE THAT CONNECTS ATC_CODE WITH DRUG NAME? -
# https://www.genome.jp/kegg-bin/get_htext#E587
drug_atc.head(30)

,stitch_code,atc_code
0,CID100000085,A16AA01
1,CID100000119,L03AA03
2,CID100000119,N03AG03
3,CID100000137,L01XD04
4,CID100000143,V03AF03
5,CID100000143,V03AF04
6,CID100000143,V03AF06
7,CID100000158,G02AD02
8,CID100000159,B01AC09
9,CID100000160,G02AD01


In [26]:
#Import chemical substr dataset


In [27]:
#Load all the Pauwel's dataset
#http://members.cbio.mines-paristech.fr/~yyamanishi/side-effect/

In [28]:
#Load Liu dataset from .mat file in NSS/capstone/data folder

In [29]:
#Load Mituzani datasets
#http://web.kuicr.kyoto-u.ac.jp/supp/smizutan/target-effect/

### Data Exploration


In [30]:
#Import off_sides dataset. OFFSIDES- associations before drug approval
off_sides=pd.read_csv('data/OFFSIDES.csv.xz.csv', compression='xz', header=0, sep=',', quotechar='"')

/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (0,2,4,5,6,7,8,9,10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [31]:
off_sides.head()

,drug_rxnorn_id,drug_concept_name,condition_meddra_id,condition_concept_name,A,B,C,D,PRR,PRR_error,mean_reporting_frequency
0,4024,"ergoloid mesylates, USP",10002034,Anaemia,6,126,21,1299,2.85714,0.45382,0.0454545
1,4024,"ergoloid mesylates, USP",10002965,Aplasia pure red cell,1,131,1,1319,10,1.41126,0.00757576
2,4024,"ergoloid mesylates, USP",10013442,Disseminated intravascular coagulation,1,131,6,1314,1.66667,1.07626,0.00757576
3,4024,"ergoloid mesylates, USP",10023126,Jaundice,2,130,7,1313,2.85714,0.79657,0.0151515
4,4024,"ergoloid mesylates, USP",10016288,Febrile neutropenia,1,131,5,1315,2,1.09163,0.00757576


In [32]:
#Import two sides dataset. TWOSIDES - data of side effects of pairs of drugs
two_sides=pd.read_csv('data/TWOSIDES.csv.xz.csv', compression='xz', header=0, sep=',', quotechar='"')

/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (0,2,4,6,7,8,9,10,11,12) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [33]:
two_sides.head()

,drug_1_rxnorn_id,drug_1_concept_name,drug_2_rxnorm_id,drug_2_concept_name,condition_meddra_id,condition_concept_name,A,B,C,D,PRR,PRR_error,mean_reporting_frequency
0,10355,Temazepam,136411,sildenafil,10003239,Arthralgia,7,149,24,1536,2.91667,0.421275,0.0448718
1,1808,Bumetanide,7824,Oxytocin,10003239,Arthralgia,1,13,2,138,5.0,1.19224,0.0714286
2,221147,POLYETHYLENE GLYCOL 3350,5521,Hydroxychloroquine,10003239,Arthralgia,6,103,20,1070,3.0,0.454505,0.0550459
3,10324,Tamoxifen,8640,Prednisone,10012735,Diarrhoea,18,123,35,1375,5.14286,0.276271,0.12766
4,10355,Temazepam,136411,sildenafil,10012735,Diarrhoea,2,154,37,1523,0.540541,0.721093,0.0128205


how many drug-drug interactions

q to michael:  two_sides have side effect for the 2 drugs already. how are we going to merge the chemical substru

In [37]:
side_effect_binary=pd.read_csv('data/source_codes_and_datasets_2015-09-05/Liu_dataset_and_experiments/Liu_dataset/merged_data/random_group_cv_data.indication', sep = '\t', delimiter = '|', engine= 'python', header = None)

In [39]:
side_effect_binary.columns=['drugBankID', 'drugName', 'compoundID', 'ADE_str', 'chem_str', \
                           'target_gene', 'transporter', 'enzyme', 'kgg', 'indication', \
                           'group_str']

In [40]:
side_effect_binary.head()

,drugBankID,drugName,compoundID,ADE_str,chem_str,target_gene,transporter,enzyme,kgg,indication,group_str
0,DB00220,nelfinavir,4451,0010000000000000000000000000000000001000000000...,0000111111111110000111000000000000000010000000...,0000000000000000000000000000000000000000000000...,0000000000000000100001000000000000000000000000...,0000000000000000000000000000000000000000010000...,0000000000000000000000000000000000000000000000...,0000000000000000000000000000000100000000000000...,"3,0,0,2,4,2,3,4,0,3,2,0,4,2,3,3,3,0,2,3,4,4,4,..."
1,DB01340,cilazapril,2751,0010000000000000000100000001000000001010000000...,0000011111011110000111000000000000000000000000...,0000000000000000000000000000000000000000000000...,0000000000000000000000000000000000000000000000...,0000000000000000000000000000000000000000000000...,0000000000000000000000000000000000000000010000...,0000000000000000000000000000000000000000000000...,"4,2,3,4,4,2,1,4,0,4,4,4,2,2,3,0,2,0,0,3,3,1,3,..."
2,DB01595,nitrazepam,4506,0000000000000000000101000000000000000000000000...,0000001111001110000011000000000000000000000000...,0000000000000000000000000001111000000000000000...,0000000000000000000000000000000000000000000000...,0000000000000000000000000000000000000000000000...,0000000000000000000000000000000000000000000000...,0000000000000000000000000000000000000000000000...,"4,4,2,0,1,1,3,2,3,3,0,1,0,3,3,4,1,4,3,4,3,0,1,..."
3,DB00900,didanosine,3043,0010000010000000000000000001000000001000000000...,0000001111001110000011010000000000000000000000...,0000000000000000000000000000000000000000000000...,0000000000000000000000000000000000000000000000...,0000000000000000000000000000000000000000000000...,0000000000001000000000000000000000000000000000...,0000000000000000000000000000000100000000000000...,"4,4,2,2,0,0,0,2,2,4,2,4,1,2,1,0,4,3,0,4,4,2,2,..."
4,DB01062,oxybutynin,4634,0010000000000000000000000000000010000000000000...,0000011101011110000011000000000000000000000000...,0000000000000000000000000000000000000000000000...,0000000000000000000000000000000000000000000000...,0000000000000000000000000000000000000000000000...,0000000000000000000000000000000100100000000000...,0000000000000000000000000000000000000000000000...,"2,0,4,0,3,1,2,4,0,2,4,1,4,2,0,3,4,2,1,4,3,0,3,..."


In [ ]:
#remove group_str from side_effect_binary

In [ ]:
#merge side_effect_binary with drug_meddra

ADE= adverse drug event

In [ ]:
#merge TWO_sides with side_effect_binary data

### Load DrugBank dataset if access provided

In [ ]:
#import DrugBank dataset
import xml.etree.ElementTree as et 

In [ ]:
xtree = et.parse("data/fu")
pd.read_table('data/')

In [ ]:
def parse_XML(xml_file, df_cols): 
    """Parse the input XML file and store the result in a pandas 
    DataFrame with the given columns. 
    
    The first element of df_cols is supposed to be the identifier 
    variable, which is an attribute of each node element in the 
    XML data; other features will be parsed from the text content 
    of each sub-element. 
    """
    
    xtree = et.parse(xml_file)
    xroot = xtree.getroot()
    rows = []
    
    for node in xroot: 
        res = []
        res.append(node.attrib.get(df_cols[0]))
        for el in df_cols[1:]: 
            if node is not None and node.find(el) is not None:
                res.append(node.find(el).text)
            else: 
                res.append(None)
        rows.append({df_cols[i]: res[i] 
                     for i, _ in enumerate(df_cols)})
    
    out_df = pd.DataFrame(rows, columns=df_cols)
        
    return out_df

In [ ]:
parse_XML("data/full_drugbank_database.xml", ['name', 'id', 'description', 'cas-number'])